In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import cv2
from tqdm import tqdm_notebook as tqdm
import pyemd

from itertools import combinations
from itertools import chain
from IPython.display import display
from scipy.spatial import distance_matrix

## Generation of Datasets
Two datasets of 500 and 7300 samples
Each person in the datasets has 6 protected attributes:
* Gender                  = {Male, Female}
* Country                 = {America, India, Other}
* Year of Birth           = [1950, 2009]
* Language                = {English, Indian, Other}
* Ethnicity               = {White, African-American, Indian, Other}
* Years of Experience     = [0,30]

And two observed attributes:
* Language Test = [25,100]
* Approval rate = [25,100]

Task Qualification Function:

$f = \alpha b_1 + (1-\alpha)b_2$

Where $b_1$ is the *language test* and $b_2$ is *approval rate* and the $\alpha \in \{0,0.3,0.5,0.7,1\}$

In [2]:
# the protected columns
protected_attrs = {
    'gender' : ['male', 'female'],
    'country' : ['america', 'india', 'other'],
    'year_birth' : (1950, 2009),
    'language' : ['english', 'india', 'other'],
    'ethnicity' : ['white', 'african-american', 'indian', 'other'],
    'year_experience' : (0,30)
}
# the observed columns
observed_attrs = {
    'language_test' : (25,100),
    'approval_rate' : (25,100)
}

In [3]:
def generate_dataset(n):
    '''Generates the dataset accordinly the parameter n that represents the amount of people'''
    # define the dataset structure
    dataset = []
    # generate the samples
    for i in range(n):
        sample_protected = [v[random.randint(0,len(v)-1)] if type(v) is list else random.randint(v[0], v[1]) for k,v in protected_attrs.items()]
        sample_observed  = [random.randint(v[0], v[1]) for k,v in observed_attrs.items()]
        sample = sample_protected + sample_observed
        dataset.append(sample)
        
    columns = list(protected_attrs.keys()) + list(observed_attrs.keys())
    return pd.DataFrame(dataset, columns=columns)

In [4]:
small_dataset = generate_dataset(500)

In [5]:
display(small_dataset)

,gender,country,year_birth,language,ethnicity,year_experience,language_test,approval_rate
0,female,india,1958,english,white,23,96,74
1,male,india,1993,other,white,26,29,90
2,female,india,1996,english,white,20,83,87
3,female,america,1958,english,white,10,63,57
4,female,other,1995,india,indian,5,70,28
5,female,india,1990,other,african-american,15,85,49
6,female,america,1961,other,other,17,89,73
7,male,other,1999,india,other,13,33,45
8,female,other,1963,english,white,14,82,51
9,male,india,1967,other,white,1,65,97


# The algorithm

In [47]:
class BalancedAlgorithm:
    def __init__(self, attributes, bins=np.arange(0,1.1,0.1)):
        self.attributes = attributes.copy()
        self.bins = bins
        self.dist = cv2.DIST_L2
        
    def generate_signature(self, h, b):
        ''''
        Convert numpy histogram in signature data structure necessary for the usage of OpenCV EMD
        Create a matrix that each row is a frequency value obtained by the histogram algorithm and the bin value (position)
        '''
        return np.array([(n, i) for i,n in enumerate(h)]).astype(np.float32)
    
    def generate_distance_matrix(self, p1, p2):
        return distance_matrix(p1.reshape(len(p1),-1), p2.reshape(len(p1),-1))
    
    def emd_pairwise(self, histograms):
        pairs = combinations(histograms, 2)
        emd_list = []
        for pair in pairs:
            distance = self.generate_distance_matrix(pair[0][1], pair[1][1])
            distance = distance / np.max(distance)
            emd_value = pyemd.emd(pair[0][0], pair[1][0], distance)
            emd_list.append(emd_value)
            
        return emd_list
    
    def generate_histogram(self, f, partition):
        samples = [f(row) for _,row in partition.iterrows()]
        h,b = np.histogram(samples, bins=self.bins)
        h = h / np.sum(h) # normalizing
        return h,b
        
    def worst_attribute(self,dataset,f,A):
        worst_attr = ''
        highest_emd = float('-inf')
        splittable = None
        
        if len(dataset) == 0:
            raise ValueError("The dataset can not be empty")
        
        for W in dataset:
            for column, possible_values in A.items():
                
                if type(possible_values) is not list:
                    possible_values = list(range(possible_values[0], possible_values[1]+1))
                
                histograms = []
                for value in possible_values:
                    query_string = '{} == "{}"'.format(column, value)
                    partition = W.query(query_string) # query by attribute value
                    
                    if partition.empty:
                        continue

                    histograms.append(self.generate_histogram(f, partition))

                # we need more than 1 attr-value to compare the histograms
                if len(histograms) <= 1:
                    continue
                
                # we need to make the pairwise EMD
                emd_list = self.emd_pairwise(histograms)

                avg_emd = np.average(emd_list)
                if avg_emd > highest_emd:
                    highest_emd = avg_emd
                    worst_attr = column
                    splittable = W
        
        assert(worst_attr is not '' and highest_emd is not float('-inf'))
        
        return worst_attr, highest_emd, splittable
        
    def split(self,W,a):
        if type(W) is list:
            array = []
            for w in W:
                array += [df for _, df in w.groupby(a)]
            return array
                
        return [df for _, df in W.groupby(a)]

    def average_emd(self,W,f):
        histograms = []
        emd_list = []
        for partition in W:
            histograms.append(self.generate_histogram(f, partition))

        if len(histograms) <= 1:
            return 0
        
        emd_list = self.emd_pairwise(histograms)
        return np.average(emd_list)

    def run(self,W,f,attr):
        removal_list = []
        avg_list = []
        A = attr.copy()
        
        a, emd_val, splittable = self.worst_attribute([W],f,A)
        
        removal_list.append(a)
        A.pop(a) # line 2 of the pseudo code
        
        current = self.split(splittable, a)
        current_avg = self.average_emd(current, f)
        avg_list.append(current_avg)

        while len(A) > 0:
            try:
                worst = self.worst_attribute(current,f,A)
            except:
                print("There is no possible to get the worst attribute with the current variable.")
                print("Current has a lenght of {}".format(len(current)))
                for c in current:
                    display(c)
            
            a = worst[0]
            A.pop(a)
            children = self.split(worst[2],a)
            
            # add the others partitions not splitted
            for partition in current:
                if partition.equals(worst[2]):
                    continue
                children += [partition]
            
            children_avg = self.average_emd(children,f)
            if current_avg >= children_avg:
                break
            else:
                current = children
                current_avg = children_avg
                
                avg_list.append(current_avg)
                removal_list.append(a)

        return current, np.mean(avg_list), removal_list, avg_list

In [48]:
class ScoringFunction:
    def __init__(self, alpha=0, b1_name='', b2_name=''):
        self.a = alpha
        self.b1_name = b1_name
        self.b2_name = b2_name
        
    def f(self,row):
        b1 = row[self.b1_name]
        b2 = row[self.b2_name]
        return (self.a*b1 + (1-self.a)*b2)

In [49]:
alpha = [0.0,0.3,0.5,0.7,1.0]

f1 = ScoringFunction(alpha=alpha[1], b1_name='language_test', b2_name='approval_rate').f
f2 = ScoringFunction(alpha=alpha[2], b1_name='language_test', b2_name='approval_rate').f
f3 = ScoringFunction(alpha=alpha[3], b1_name='language_test', b2_name='approval_rate').f
f4 = ScoringFunction(alpha=alpha[4], b1_name='language_test', b2_name='approval_rate').f
f5 = ScoringFunction(alpha=alpha[0], b1_name='language_test', b2_name='approval_rate').f

f6 = lambda row: random.uniform(.8, 1) if row['gender'] == 'male' else random.uniform(0, .2)

In [55]:
r1 = []
r2 = []
r3 = []
r4 = []
r5 = []
r6 = []
for i in tqdm(range(10)):
    balanced = BalancedAlgorithm(protected_attrs, bins=np.arange(25,102))
    balanced2 = BalancedAlgorithm(protected_attrs)
    
    result1 = balanced.run(small_dataset.copy(), f1, protected_attrs)
    result2 = balanced.run(small_dataset.copy(), f2, protected_attrs)
    result3 = balanced.run(small_dataset.copy(), f3, protected_attrs)
    result4 = balanced.run(small_dataset.copy(), f4, protected_attrs)
    result5 = balanced.run(small_dataset.copy(), f5, protected_attrs)
    result6 = balanced2.run(small_dataset.copy(), f6, protected_attrs)
    
    small_dataset = generate_dataset(500)
    
    r1.append(result1[1])
    r2.append(result2[1])
    r3.append(result3[1])
    r4.append(result4[1])
    r5.append(result5[1])
    r6.append(result6[1])

In [56]:
print("F1 Average EMD = {}".format(np.average(r1)))
print("F2 Average EMD = {}".format(np.average(r2)))
print("F3 Average EMD = {}".format(np.average(r3)))
print("F4 Average EMD = {}".format(np.average(r4)))
print("F5 Average EMD = {}".format(np.average(r5)))
print("F6 Average EMD = {}".format(np.average(r6)))

F1 Average EMD = 0.15788375847645222
F2 Average EMD = 0.148685846121797
F3 Average EMD = 0.1544794321949134
F4 Average EMD = 0.19025563226842085
F5 Average EMD = 0.19214884813600971
F6 Average EMD = 0.79957418042592
